### Imports

In [ ]:
import os
import pandas as pd
import numpy as np
from functools import reduce


In [ ]:
data_dir = "../base_data"
new_data_dir = "new_data"

In [ ]:
columns_to_keep_matches = {
    'fixtures.csv': ['eventId', 'homeTeamId', 'awayTeamId','leagueId', 'homeTeamWinner', 'awayTeamWinner', 'homeTeamScore', 'awayTeamScore', 'date'],
    'teamStats.csv': ['eventId', 'teamId', 'possessionPct', 'totalShots', 'shotsOnTarget', 'saves', 'wonCorners'],
}

In [ ]:
dfs_matches = []

for filename, columns in columns_to_keep_matches.items():
    path = os.path.join(data_dir, filename)
    df = pd.read_csv(path, usecols=columns)
    dfs_matches.append(df)
   

In [ ]:
merged_df_matches = reduce(lambda left, right: pd.merge(left, right, on='eventId', how='inner'), dfs_matches)

# Generate ordered column list automatically
ordered_columns_m = []
for cols in columns_to_keep_matches.values():
    for col in cols:
        if col not in ordered_columns_m:
            ordered_columns_m.append(col)

# Reorder columns safely
merged_df_matches = merged_df_matches[[col for col in ordered_columns_m if col in merged_df_matches.columns]]
merged_df_matches = merged_df_matches.dropna()

In [ ]:
target_leagues = [720, 740, 700, 730]
merged_df_matches = merged_df_matches[merged_df_matches['leagueId'].isin(target_leagues)]
merged_df_matches

In [ ]:
cols_to_int = ['totalShots','shotsOnTarget','saves','wonCorners']
merged_df_matches[cols_to_int] = merged_df_matches[cols_to_int].astype(int)

In [ ]:
merged_df_matches = merged_df_matches.groupby(['eventId']).agg(
homeTeamId=pd.NamedAgg(column="homeTeamId", aggfunc="first"),
awayTeamId=pd.NamedAgg(column="awayTeamId", aggfunc="first"),
leagueId=pd.NamedAgg(column="leagueId", aggfunc="first"),
homeTeamScore=pd.NamedAgg(column="homeTeamScore", aggfunc="first"),
awayTeamScore=pd.NamedAgg(column="awayTeamScore", aggfunc="first"),
homeTeamWinner=pd.NamedAgg(column="homeTeamWinner", aggfunc="first"),
awayTeamWinner=pd.NamedAgg(column="awayTeamWinner", aggfunc="first"),
date=pd.NamedAgg(column="date", aggfunc="first"),
possessionPct_home=pd.NamedAgg(column="possessionPct", aggfunc="first"),
possessionPct_away=pd.NamedAgg(column="possessionPct", aggfunc="last"),
totalShots_home=pd.NamedAgg(column="totalShots", aggfunc="first"),
totalShots_away=pd.NamedAgg(column="totalShots", aggfunc="last"),
shotsOnTarget_home=pd.NamedAgg(column="shotsOnTarget", aggfunc="first"),
shotsOnTarget_away=pd.NamedAgg(column="shotsOnTarget", aggfunc="last"),
saves_home=pd.NamedAgg(column="saves", aggfunc="first"),
saves_away=pd.NamedAgg(column="saves", aggfunc="last"),
wonCorners_home=pd.NamedAgg(column="wonCorners", aggfunc="first"),
wonCorners_away=pd.NamedAgg(column="wonCorners", aggfunc="last"),

).sort_values(by=['leagueId']).reset_index()  # ✅ Sort after aggregation.reset_index()

In [ ]:
output_path_matches = os.path.join(new_data_dir, 'database_matches.csv')

os.makedirs(os.path.dirname(output_path_matches), exist_ok=True)

merged_df_matches.to_csv(output_path_matches, index=False)

print(f"Merged CSV saved to {output_path_matches}")
